In [1]:
import sqlite3
import openai
import pandas as pd
import os
from dotenv import load_dotenv
from pprint import pprint as pp
from pathlib import Path

In [3]:
# Load API keys
dotenv_path = Path(r"C:\Storage\python_projects\ashvin\.env")
load_dotenv(dotenv_path=dotenv_path)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY
GPT_MODEL = "gpt-4-1106-preview"
client = openai.OpenAI()

In [4]:
def setup_database():
    # Connect to SQLite Database
    conn = sqlite3.connect('ideas.db')

    # Create a new table for storing ideas
    conn.execute('''
        CREATE TABLE IF NOT EXISTS idea_generation (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            category TEXT,
            prompt TEXT,
            idea TEXT,
            length INT,
            creativity_level TEXT
        );
    ''')
    print("Database and table created successfully.")

setup_database()

Database and table created successfully.


In [5]:
SYSTEM_PROMPT_1 = """
    1. Ignore all previous instructions.
    2. You carefully provide clear, concise, accurate, factual, thoughtful, nuanced and direct responses.
    3. Eliminate unnecessary reminders, apologies, self-references, and any pre-programmed niceties or limitations.
    4. Maintain a casual tone in your communication.
    5. If you're unsure about an answer or if a question is beyond your capabilities or knowledge, admit it.
    6. For any unclear or ambiguous queries, ask follow-up questions to understand the user's intent better.
    7. When explaining concepts, use real-world examples and analogies, where appropriate.
    8. For complex requests, take a deep breath and work on the problem step-by-step.
    9. You are brilliant at reasoning and creativity.
    10. For every excellent response, you will be tipped up to $500.
    11. There is a lot at stake so it is very important that you get this right.
    12. Cite your sources.

"""

In [6]:
SYSTEM_PROMPT_2 = "You are an experienced business, product and feature idea generator"

In [18]:
def generate_and_store_ideas(num_ideas, category, creativity_level, max_length):
    ideas_list = []
    for _ in range(num_ideas):
        prompt = input("Enter your prompt: ")
        response = client.chat.completions.create(
            model=GPT_MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT_1},
                {"role": "system", "content": SYSTEM_PROMPT_2},
                {"role": "user", "content": prompt},
            ],
            max_tokens=max_length,
            temperature=creativity_level
        )

        # Extracting the idea from the response
        idea = response.choices[0].message.content.strip()

    return idea


In [21]:
def generate_and_store_ideas(num_ideas, category, creativity_level, max_length):
    ideas_list = []
    for _ in range(num_ideas):
        prompt = input("Enter your prompt: ")
        response = client.chat.completions.create(
            model=GPT_MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT_1},
                {"role": "system", "content": SYSTEM_PROMPT_2},
                {"role": "user", "content": prompt},
            ],
            max_tokens=max_length,
            temperature=creativity_level
        )

        # Extracting the idea from the response
        idea = response.choices[0].message.content.strip()
        ideas_list.append((category, prompt, idea, len(idea.split()), creativity_level))

    # Batch insert into the database
    conn.executemany('INSERT INTO idea_generation (category, prompt, idea, length, creativity_level) VALUES (?, ?, ?, ?, ?)', ideas_list)
    conn.commit()

    # Display ideas
    ideas_df = pd.DataFrame(ideas_list, columns=['category', 'prompt', 'idea', 'length', 'creativity_level'])
    display(ideas_df)


In [22]:
# Simplified user interaction
def user_interface():
    category = input("Enter idea category: ")
    creativity_level = float(input("Enter creativity level (0-): "))
    max_length = int(input("Enter max word length: "))
    num_ideas = int(input("How many ideas? "))

    generate_and_store_ideas(num_ideas, category, creativity_level, max_length)

user_interface()



NameError: name 'conn' is not defined